In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, Model
import pickle   


2025-07-10 20:08:29.529709: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752192509.550248 4181949 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752192509.556592 4181949 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1752192509.572332 4181949 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1752192509.572349 4181949 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1752192509.572352 4181949 computation_placer.cc:177] computation placer alr

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [3]:
import numpy as np

captions_path = "./padded_sequences.npz"        
captions = np.load(captions_path, allow_pickle=True)
caption_data = {img_id: captions[img_id] for img_id in captions.files}

image_features_path = "./flickr8k_features.npz" 
features_npz = np.load(image_features_path, allow_pickle=True)
image_features = {k: features_npz[k] for k in features_npz.files}


In [5]:
with open('./flickr8k_tokenizer.pkl', 'rb') as f:
    tokenizer = pickle.load(f)


captions_path = "/Users/desireereid/Downloads/padded_sequences.npz"
captions = np.load(captions_path, allow_pickle=True)
caption_data = {img_id: captions[img_id] for img_id in captions.files} -- if you're running locally. 

In [6]:
EMBED_DIM = 512
UNITS = 512

VOCAB_SIZE = len(tokenizer.word_index) + 1   
MAX_LEN = max(len(seq) for seqs in caption_data.values() for seq in seqs)                     

**Note:** The values for `VOCAB_SIZE` and `MAX_LEN` are placeholders for now.  
- `VOCAB_SIZE = 5000` is a rough estimate until the tokenizer is available.  
- `MAX_LEN = 34` is a reasonable max caption length based on the dataset.  
Once the tokenizer is loaded, update:
- `VOCAB_SIZE = len(tokenizer.word_index) + 1`
- `MAX_LEN` = actual max length of the padded captions


In [7]:
class ImageCaptioningModel(tf.keras.Model):
    def __init__(self, embed_dim, units, vocab_size):
        super().__init__()
        self.embedding  = layers.Embedding(vocab_size, embed_dim)
        self.image_proj = layers.Dense(units, activation='relu')
        self.lstm       = layers.LSTM(units, return_sequences=True, return_state=True)
        self.fc         = layers.Dense(vocab_size)

    def call(self, inputs):
        img_feat, captions = inputs          
        img_proj = self.image_proj(img_feat) 
        img_proj = tf.expand_dims(img_proj, 1)

        cap_emb  = self.embedding(captions)  
        lstm_in  = tf.concat([img_proj, cap_emb], axis=1)

        lstm_out, _, _ = self.lstm(lstm_in)
        return self.fc(lstm_out)             


In [8]:
model = ImageCaptioningModel(
    embed_dim=EMBED_DIM,
    units=UNITS,
    vocab_size=VOCAB_SIZE
) 

2025-07-10 20:13:18.500592: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2025-07-10 20:13:18.501486: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:167] env: CUDA_VISIBLE_DEVICES="-1"
2025-07-10 20:13:18.501499: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:170] CUDA_VISIBLE_DEVICES is set to -1 - this hides all GPUs from CUDA
2025-07-10 20:13:18.501508: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:178] verbose logging is disabled. Rerun with verbose logging (usually --v=1 or --vmodule=cuda_diagnostics=1) to get more diagnostic output from this module
2025-07-10 20:13:18.501513: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:183] retrieving CUDA diagnostic information for host: c2190
2025-07-10 20:13:18.501517: I external/local_xla/xla/stream_executor/cuda/cuda_di

In [9]:
sample_img     = list(image_features.values())[0].reshape(1, -1)       
sample_caption = np.array([caption_data[list(caption_data.keys())[0]][0]]) 

model = ImageCaptioningModel(embed_dim=EMBED_DIM,
                             units=UNITS,
                             vocab_size=VOCAB_SIZE)

output = model((sample_img, sample_caption))  
print("Output shape:", output.shape)           

Output shape: (1, 20, 8481)


 #### Build a tf.data.Dataset 

In [10]:
import tensorflow as tf

def make_dataset(img_feat_dict, cap_dict, batch_size=32, shuffle=True):
    img_ids = list(img_feat_dict.keys())               
    img_feats = [img_feat_dict[i] for i in img_ids]      
    caps      = [cap_dict[i]      for i in img_ids]      

    flat_feats = []
    flat_caps  = []
    for f, five_caps in zip(img_feats, caps):
        for c in five_caps:
            flat_feats.append(f)
            flat_caps.append(c)

    ds = tf.data.Dataset.from_tensor_slices((flat_feats, flat_caps))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(flat_feats))
    ds = ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return ds

train_ds = make_dataset(image_features, caption_data, batch_size=64)


#### Loss & Optimizer

In [14]:
loss_fn   = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()

def masked_loss(y_true, y_pred):
    """
    Ignore padding-token positions (0) when computing loss.
    """
    mask  = tf.cast(tf.not_equal(y_true, 0), tf.float32)
    loss  = loss_fn(y_true, y_pred) * mask
    return tf.reduce_sum(loss) / tf.reduce_sum(mask)


#### One Traning Step 

In [15]:
@tf.function
def train_step(img_batch, cap_batch):
    """
    img_batch : (B, 2048)
    cap_batch : (B, MAX_LEN)
    
    We feed caption[:-1] as input and predict caption[1:].
    """
    inp  = cap_batch[:, :-1]   
    targ = cap_batch[:, 1:]    

    with tf.GradientTape() as tape:
        logits = model((img_batch, inp))     

        logits = logits[:, 1:, :]           

        loss   = masked_loss(targ, logits)

    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    return loss


#### Small Traning Loop 

In [16]:
EPOCHS = 2          
STEPS  = 100        
for epoch in range(EPOCHS):
    total, steps = 0.0, 0

    for img_b, cap_b in train_ds.take(STEPS):
        batch_loss = train_step(img_b, cap_b)   
        total += float(batch_loss)              
        steps += 1

    print(f"Epoch {epoch+1} – avg loss: {total/steps:.4f}")


2025-07-10 20:20:49.465374: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 1 – avg loss: 9.0476
Epoch 2 – avg loss: 8.9786


2025-07-10 20:20:50.182777: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


#### Full Training Loop w/patience + checkpointing

In [33]:
import os, pathlib, datetime, pickle, numpy as np, tensorflow as tf
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"   


In [34]:
# image feature vectors
feat_path = pathlib.Path("./flickr8k_features.npz")
features_npz = np.load(feat_path, allow_pickle=True)
image_features = {k: features_npz[k] for k in features_npz.files}

# padded caption sequences
cap_path = pathlib.Path("./padded_sequences.npz")
caps_npz = np.load(cap_path, allow_pickle=True)
caption_data = {k: caps_npz[k] for k in caps_npz.files}

# tokenizer 
with open("./flickr8k_tokenizer.pkl", "rb") as f:
    tokenizer = pickle.load(f)

VOCAB_SIZE = len(tokenizer.word_index) + 1          
MAX_LEN    = next(iter(caption_data.values()))[0].shape[-1] 


In [35]:
print(len(image_features), "images   |   max-len =", MAX_LEN)


10 images   |   max-len = 19


In [36]:
import tensorflow as tf
import numpy as np

BATCH = 4          # set 32-64 once we have the full dataset
BUFFER = 8_000

def make_dataset(img_dict, cap_dict):
    """Yields (img_vec, caption_in)  → caption_out."""
    def gen():
        for img_id, feats in img_dict.items():
            for cap in cap_dict[img_id]:
                yield feats.astype("float32"), np.array(cap, dtype="int32")

    raw_ds = tf.data.Dataset.from_generator(
        gen,
        output_signature=(
            tf.TensorSpec(shape=(2048,),    dtype=tf.float32),  
            tf.TensorSpec(shape=(MAX_LEN,), dtype=tf.int32)    
        )
    )

    def prep(feats, full_cap):
        return (feats, full_cap[:-1]), full_cap[1:]

    return (raw_ds
              .shuffle(BUFFER)
              .map(prep, num_parallel_calls=tf.data.AUTOTUNE)
              .batch(BATCH, drop_remainder=True)        
              .prefetch(tf.data.AUTOTUNE))

train_ds = make_dataset(image_features, caption_data)
print("example batch shapes:",
      next(iter(train_ds.take(1)))[0][0].shape,   
      next(iter(train_ds.take(1)))[0][1].shape,  
      next(iter(train_ds.take(1)))[1].shape)      


example batch shapes: (4, 2048) (4, 18) (4, 18)


In [40]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(
            from_logits=True, reduction="none")

def masked_loss(y_true, y_pred):
    """
    y_true : (B, T)           – 18 tokens
    y_pred : (B, T+1, V)      – 19 logits (image step + T tokens)
    We remove the first step so that time-dims match.
    """
    y_pred = y_pred[:, 1:, :]          # <--- crop image timestep
    mask   = tf.cast(tf.not_equal(y_true, 0), tf.float32)

    loss   = loss_fn(y_true, y_pred) * mask
    return tf.reduce_sum(loss) / tf.reduce_sum(mask)


In [41]:
model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
              loss=masked_loss)

history = model.fit(
    train_ds,
    epochs=EPOCHS,
    callbacks=[ckpt_cb, early_cb, tb_cb]
)


Epoch 1/15
     11/Unknown 4s 163ms/step - loss: 8.5292

/home/reid.des/.local/lib/python3.12/site-packages/keras/src/trainers/epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


11/11 ━━━━━━━━━━━━━━━━━━━━ 5s 197ms/step - loss: 8.4009
Epoch 2/15
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 197ms/step - loss: 4.7350
Epoch 3/15
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 167ms/step - loss: 4.3368

2025-07-10 20:43:31.153379: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 201ms/step - loss: 4.3091
Epoch 4/15
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 191ms/step - loss: 4.2096
Epoch 5/15
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 188ms/step - loss: 4.1474
Epoch 6/15
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 200ms/step - loss: 4.0147
Epoch 7/15
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 194ms/step - loss: 3.8208
Epoch 8/15
11/11 ━━━━━━━━━━━━━━━━━━━━ 3s 192ms/step - loss: 3.7162
Epoch 9/15
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 194ms/step - loss: 3.6558
Epoch 10/15
11/11 ━━━━━━━━━━━━━━━━━━━━ 3s 192ms/step - loss: 3.5262
Epoch 11/15
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 160ms/step - loss: 3.4196

2025-07-10 20:43:48.994611: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 193ms/step - loss: 3.3914
Epoch 12/15
11/11 ━━━━━━━━━━━━━━━━━━━━ 3s 202ms/step - loss: 3.2273
Epoch 13/15
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 198ms/step - loss: 3.1843
Epoch 14/15
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 194ms/step - loss: 3.0316
Epoch 15/15
11/11 ━━━━━━━━━━━━━━━━━━━━ 3s 194ms/step - loss: 2.8535


In [42]:
with open('tokenizer.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)

np.savez('training_meta.npz', max_len=MAX_LEN, vocab_size=VOCAB_SIZE)


In [44]:
model = ImageCaptioningModel(EMBED_DIM, UNITS, VOCAB_SIZE)


dummy_img  = tf.zeros((1, 2048), dtype=tf.float32)   # (B, 2048)
dummy_cap  = tf.zeros((1, MAX_LEN), dtype=tf.int32)  # (B, MAX_LEN)
_ = model((dummy_img, dummy_cap))                    # build graph


model.load_weights("checkpoints/best.weights.h5")
print("Weights loaded ✅")


Weights loaded ✅


In [45]:
model = ImageCaptioningModel(EMBED_DIM, UNITS, VOCAB_SIZE)

model.build(input_shape=[(None, 2048), (None, MAX_LEN)])

model.load_weights("checkpoints/best.weights.h5")
print("Weights loaded ✅")


Weights loaded ✅


/home/reid.des/.local/lib/python3.12/site-packages/keras/src/layers/layer.py:396: UserWarning: `build()` was called on layer 'image_captioning_model_6', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


In [46]:
sample_img = list(image_features.values())[0].reshape(1, -1)
start_seq  = [tokenizer.word_index.get("<start>", 1)]
caption_in = tf.constant(start_seq + [0]*(MAX_LEN-1))[None, :]

logits = model((sample_img, caption_in))
print("Output logits shape:", logits.shape)   


Output logits shape: (1, 20, 8481)


In [48]:
import tensorflow as tf
import numpy as np

one_img_id      = next(iter(caption_data))           # pick any key
one_caption_seq = caption_data[one_img_id][0]        # first caption list
START_ID        = int(one_caption_seq[0])            # first token in every seq
END_ID          = int(one_caption_seq[-1])           # last token (padding-aware)

print(f"START_ID = {START_ID}  |  END_ID = {END_ID}")


def greedy_caption(model, img_vec, tokenizer,
                   start_id=START_ID, end_id=END_ID,
                   max_len=one_caption_seq.shape[-1]):
    """
    Args
        img_vec : (1, 2048) float32 numpy / tensor
        tokenizer : fitted Keras tokenizer
    Returns
        string  – generated caption (without <start>/<end>)
    """
    caption_ids = [start_id]                         
    for _ in range(max_len - 1):
        inp = tf.constant([caption_ids + [0]*(max_len - len(caption_ids))],
                          dtype=tf.int32)
        logits = model((img_vec, inp))[:, 1:, :]     
        next_id = int(tf.argmax(logits[0, len(caption_ids)-1]).numpy())

        if next_id in (end_id, 0):                   
            break
        caption_ids.append(next_id)

    inv_vocab = {i:w for w,i in tokenizer.word_index.items()}
    words = [inv_vocab.get(i, "<unk>") for i in caption_ids[1:]]  
    return " ".join(words)


sample_img_vec = list(image_features.values())[0].reshape(1, -1).astype("float32")

generated = greedy_caption(model, sample_img_vec, tokenizer)
print("\nGenerated caption:\n", generated)


START_ID = 3  |  END_ID = 4

Generated caption:
 bluestriped bluestriped bluestriped bluestriped bluestriped intensely intensely intensely sidecar accross intensely filed filed eyese eyese eyese labeled showering
